In [1]:
import sympy as sp
import numpy as np
import itertools as it
import functools as ft
import pickle
import multiprocessing as mp
mp.set_start_method('fork') # 'spawn' might be better on some platforms

In [2]:
the_graph = graphs.OrthogonalPolarGraph(6,3,'-')
num_vert = the_graph.order()

In [3]:
def coeff_func_for_part(box_size,partition,ext_ext_edges=[],ext_int_edges=[],int_int_edges=[],num_int_vert=0):
    """Constructs a coefficient function for a given partition
    
    Constructs a function that returns the coefficient of tuples in the `box_size`-th box space
    of the spin planar algebra, specified by the partition.
    
    Parameters
    ----------
    box_size : int
    partition : list of sets of integers 0 .. box_size-1. You can omit singleton sets, so that
    [{1,2}] and [{0},{1,2}] would have the same effect.
    ext_ext_edges : list of pairs (a, b)
    """

    def coeff_func(*variables):
        if len(variables) == 1 and variables[0] == "diag":
            return diag_for_4_box_elem(partition,ext_ext_edges,ext_int_edges,int_int_edges,num_int_vert)
        if len(variables) != box_size:
            raise Exception("number of variables does not match the box size.")
        for part in partition:
            vars_in_part = [variables[i] for i in range(box_size) if i in part]
            if len(set(vars_in_part)) > 1:
                return 0
        if any(True for edge in ext_ext_edges if not variables[edge[0]] in the_graph.neighbors(variables[edge[1]])):
            return 0
        if num_int_vert == 0:
            return 1
        int_vert_candidates = []
        for i in range(num_int_vert):
            ext_vert_neighs = [set(the_graph.neighbors(variables[edge[0]])) for edge in ext_int_edges if edge[1] == i]
            cand_set = ft.reduce(lambda x, y: x.intersection(y), ext_vert_neighs, set(range(num_vert)))
            int_vert_candidates.append(cand_set)
        def branch_candidates_for(prev_verts):
            i = len(prev_verts)
            prev_int_verts_conn_by_edges_0 = {edge[0] for edge in int_int_edges if edge[1] == i and edge[0] < i}
            prev_int_verts_conn_by_edges_1 = {edge[1] for edge in int_int_edges if edge[0] == i and edge[1] < i}
            prev_int_verts_conn_by_edges = prev_int_verts_conn_by_edges_0.union(prev_int_verts_conn_by_edges_1)
            nvc = [v for v in int_vert_candidates[i] if not any(True for j in prev_int_verts_conn_by_edges if not v in the_graph.neighbors(prev_verts[j]))]
            if len(prev_verts) == num_int_vert - 1:
                return [prev_verts + [v] for v in nvc]
            else:
                return [x for v in nvc for x in branch_candidates_for(prev_verts + [v])]
        valid_int_conf = branch_candidates_for([])
        return len(valid_int_conf)
    return coeff_func

In [4]:
def paths_for_4_box_elem(partition,ext_ext_edges=[],ext_int_edges=[],int_int_edges=[],num_int_vert=0):
    output =""
    if num_int_vert == 1:
        output += r'\node (inn_v0) at (0,0) [inn-vert] {};' + '\n'
    elif num_int_vert == 2:
        output += r'\node (inn_v0) at (-0.3,0) [inn-vert] {};' + '\n'
        output += r'\node (inn_v1) at (0.3,0) [inn-vert] {};' + '\n'
    elif num_int_vert == 3:
        output += r'\node (inn_v0) at (-0.4,0.3) [inn-vert] {};' + '\n'
        output += r'\node (inn_v1) at (0.4,0.3) [inn-vert] {};' + '\n'
        output += r'\node (inn_v2) at (0,-0.4) [inn-vert] {};' + '\n'
    elif num_int_vert == 4:
        output += r'\node (inn_v0) at (0,0.5) [inn-vert] {};' + '\n'
        output += r'\node (inn_v1) at (0.5,0) [inn-vert] {};' + '\n'
        output += r'\node (inn_v2) at (0,-0.5) [inn-vert] {};' + '\n'
        output += r'\node (inn_v3) at (-0.5,0) [inn-vert] {};' + '\n'
    if any({0,1}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner0) -- (corner1);' + '\n'
    if any({1,2}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner1) -- (corner2);' + '\n'
    if any({2,3}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner2) -- (corner3);' + '\n'
    if any({3,0}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner3) -- (corner0);' + '\n'
    if not any({0,1}.issubset(s) for s in partition) and not any({3,0}.issubset(s) for s in partition) and any({0,2}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner0) -- (corner2);' + '\n'    
    if not any({0,1}.issubset(s) for s in partition) and not any({1,2}.issubset(s) for s in partition) and any({1,3}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner1) -- (corner3);' + '\n'    
    for ee_edge in ext_ext_edges:
        output += r'\draw [] (corner%d) -- (corner%d);' % ee_edge + '\n'
    for ei_edge in ext_int_edges:
        output += r'\draw [] (corner%d) -- (inn_v%d);' % ei_edge + '\n'
    for ii_edge in int_int_edges:
        output += r'\draw [] (inn_v%d) -- (inn_v%d);' % ii_edge + '\n'
    return output

def diag_for_4_box_elem(partition,ext_ext_edges=[],ext_int_edges=[],int_int_edges=[],num_int_vert=0):
    output = r'\begin{tikzpicture}' + '\n'
    output += r'\draw[bgrect] (-1.3,-1.3) rectangle (1.3,1.3);' + '\n'
    output += r'\node (corner0) at (-1,1) [ext-vert] {};' + '\n'
    output += r'\node (corner1) at (1,1) [ext-vert] {};' + '\n'
    output += r'\node (corner2) at (1,-1) [ext-vert] {};' + '\n'
    output += r'\node (corner3) at (-1,-1) [ext-vert] {};' + '\n'
    output += paths_for_4_box_elem(partition,ext_ext_edges,ext_int_edges,int_int_edges,num_int_vert)
    output += r'\end{tikzpicture}' + '\n'
    return output

helper functions to get functions by rotated patterns

In [5]:
def rot_ind(x,step,length):
    y = x + step
    if y < length:
        return y
    return y-length

def rot_partition(partition,step,length):
    """Rotate partitions
    
    Parameters
    ==========
    partition : collection of sets of indexes for external leaves, singletons can be omitted
    step : int, for step of rotations
    length : int, period of rotation
    """
    return [set([rot_ind(x,step,length) for x in block]) for block in partition]

def rot_e_e_edges(edges,step,length):
    """Rotated edges between external leaves
    
    Parameters
    ==========
    edges : collection of pairs of indexes for external leaves
    step : int, for step of rotations
    length : int, period of rotation
    """
    return [(rot_ind(edge[0],step,length),rot_ind(edge[1],step,length)) for edge in edges]

def rot_e_i_edges(edges,step,length):
    """Rotated edges between external leaves and internal vertices
    
    Parameters
    ==========
    edges : collection of pairs (index for external leaves, index for interval vertices)
    step : int, for step of rotations
    length : int, period of rotation
    """
    return [(rot_ind(edge[0],step,length),edge[1]) for edge in edges]

def coeff_funcs_parts_e_e_rotated_3(partition,ee_edges):
    """Rotated coefficient functions based on edges between external leaves
    
    Parameters
    ==========
    partition : collection of sets of indexes for external leaves, singletons can be omitted
    ee_edges : collection of pairs of indexes for external leaves
    """
    return [coeff_func_for_part(3,rot_partition(partition,i,3),rot_e_e_edges(ee_edges,i,3)) for i in range(3)]
def coeff_funcs_parts_e_e_e_i_rotated_3(partition,ee_edges,ei_edges,num_int_e):
    """Rotated coefficient functions based on edges between external leaves,
    and those between external leaves and internal vertices
    
    Parameters
    ==========
    partition : collection of sets of indexes for external leaves, singletons can be omitted
    ee_edges : collection of pairs of indexes for external leaves
    ei_edges : collection of pairs (index for external leaves, index for interval vertices)
    num_int_e : int, number of internal vertices
    """
    return [coeff_func_for_part(3,rot_partition(partition,i,3),rot_e_e_edges(ee_edges,i,3),rot_e_i_edges(ei_edges,i,3),[],num_int_e) for i in range(3)]

In [6]:
nc_partitions_3 = [[],[{0,1}],[{1,2}],[{0,2}],[{0,1,2}]]

In [7]:
func_list_nc_3 = [coeff_func_for_part(3,this_nc_part) for this_nc_part in nc_partitions_3]
func_list_6_8 = coeff_funcs_parts_e_e_rotated_3([],[(0,2)])
func_list_9_11 = coeff_funcs_parts_e_e_rotated_3([{0,2}],[(0,1)])
func_list_12_14 = coeff_funcs_parts_e_e_rotated_3([],[(0,1),(1,2)])
func_list_15_16 = [coeff_func_for_part(3,[],[],[(0,0),(1,0),(2,0)],[],1),coeff_func_for_part(3,[],[(0,1),(1,2),(0,2)])]

In [8]:
A_funcs = func_list_nc_3 + func_list_6_8 + func_list_9_11 + func_list_12_14 + [func_list_15_16[1]]

get representatives of orbits for triples of vertices

In [9]:
trip_vert_orb_rep = [(0,0,0)] # start with diagonal
# 0 - neighbor - 0
neigh_0 = the_graph.neighbors(0)[0]
trip_vert_orb_rep += [(0, 0, neigh_0), (0, neigh_0, 0), (neigh_0, 0, 0)]
# 0 - neghbor - vertex at distance 2 from 0, distance 1 from neighbor
vert_set_dist_1 = set(the_graph.neighbors(0))
neighbors_neigh_0 = set(the_graph.neighbors(neigh_0))
vert_set_dist_2 = neighbors_neigh_0.difference(vert_set_dist_1).difference({0})
dist_2_fr_0 = list(vert_set_dist_2)[0]
trip_vert_orb_rep += [(0, neigh_0, dist_2_fr_0), (dist_2_fr_0, 0, neigh_0), (neigh_0, dist_2_fr_0, 0)]
# 0 - 0 - vertex at distance 2 from 0
trip_vert_orb_rep += [(0, 0, dist_2_fr_0), (0, dist_2_fr_0, 0), (dist_2_fr_0, 0, 0)]
# 0 - neighbor - vertex at distance 2 from both
unconn_vert_set = set(the_graph.vertices()).difference(vert_set_dist_1).difference(neighbors_neigh_0).difference({0, neigh_0})
unconn_vert = list(unconn_vert_set)[0]
trip_vert_orb_rep += [(0, neigh_0, unconn_vert), (0, unconn_vert, neigh_0), (unconn_vert, 0, neigh_0)]
# unconnected triple
unconn_verts_from_0_and_dist_2_fro_0 = set(the_graph.vertices()).difference(vert_set_dist_1).difference(set(the_graph.neighbors(dist_2_fr_0))).difference({0, dist_2_fr_0})
unconn_from_both = list(unconn_verts_from_0_and_dist_2_fro_0)[0]
trip_vert_orb_rep.append((0, dist_2_fr_0, unconn_from_both))
# triangle
triang_vert_set = neighbors_neigh_0.intersection(vert_set_dist_1)
if len(triang_vert_set) > 0 :
    trip_vert_orb_rep.append((0, neigh_0, list(triang_vert_set)[0]))
print(len(trip_vert_orb_rep))
trip_vert_orb_rep

15


[(0, 0, 0),
 (0, 0, 32),
 (0, 32, 0),
 (32, 0, 0),
 (0, 32, 5),
 (5, 0, 32),
 (32, 5, 0),
 (0, 0, 5),
 (0, 5, 0),
 (5, 0, 0),
 (0, 32, 1),
 (0, 1, 32),
 (1, 0, 32),
 (0, 5, 1),
 (0, 32, 64)]

check the relation in 3-box space; the 16th element in Liu's presentation should be in the span of the first 14 for this graph

In [10]:
def myfunc1(x):
    return [func(x[0],x[1],x[2]) for func in A_funcs]
def myfunc2(x):
    return func_list_15_16[0](x[0],x[1],x[2])

if __name__ == '__main__':
    with mp.Pool() as p:
        A_array = p.map(myfunc1, trip_vert_orb_rep)
        b_array = p.map(myfunc2, trip_vert_orb_rep)

In [11]:
sols = sp.linsolve((sp.Matrix(A_array),sp.Matrix(b_array))); sols

{(4, 6, 6, 6, 8, -3, -3, -3, -4, -4, -4, 2, 2, 2, 0)}

dimension of the 3-box space

In [12]:
Apr_funcs = func_list_nc_3 + func_list_6_8 + func_list_9_11 + func_list_12_14 + func_list_15_16
Apr_array = [[func(x,y,z) for func in Apr_funcs] for x, y, z in trip_vert_orb_rep]
sp.Matrix(Apr_array).rank()

15

helper functions to get rotated elements in the 4-box space

In [13]:
def coeff_funcs_parts_e_e_rotated_4(partition,ee_edges):
    return [coeff_func_for_part(4,rot_partition(partition,i,4),rot_e_e_edges(ee_edges,i,4)) for i in range(4)]
def coeff_funcs_parts_e_e_e_i_rotated_4(partition,ee_edges,ei_edges,num_int_e):
    return [coeff_func_for_part(4,rot_partition(partition,i,4),rot_e_e_edges(ee_edges,i,4),rot_e_i_edges(ei_edges,i,4),[],num_int_e) for i in range(4)]
def coeff_funcs_parts_e_e_e_i_i_i_rotated_4(partition,ee_edges,ei_edges,ii_edges,num_int_e):
    return [coeff_func_for_part(4,rot_partition(partition,i,4),rot_e_e_edges(ee_edges,i,4),rot_e_i_edges(ei_edges,i,4),ii_edges,num_int_e) for i in range(4)]

get orbits for the 4-th square of vertices

In [14]:
quad_vert_orb_rep = []
quad_vert_orb_fourth_cand = {}
for trip_orb in trip_vert_orb_rep:
    singletons = [[x] for x in set(trip_orb)]
    fourth_coord_orbits = the_graph.automorphism_group(partition=singletons+[list(set(the_graph.vertices()).difference(set(trip_orb)))],orbits=True,return_group=False)
    for orb in fourth_coord_orbits:
        quad_orb = trip_orb + (orb[0],)
        quad_vert_orb_rep += [quad_orb]
        quad_vert_orb_fourth_cand[quad_orb] = set(orb)
print(f"There are {len(quad_vert_orb_rep)} orbits in the quadruple product of vertices.")

There are 134 orbits in the quadruple product of vertices.


elements from nc partitions / TL diagrams

In [15]:
nc_partitions_4 = [[]] + [rot_partition([{0,1}],i,4) for i in range(4)] + [[{1,3}],[{0,2}]] + [rot_partition([{0,1,2}],i,4) for i in range(4)] + [[{0,1,2,3}],[{0,1},{2,3}],[{0,3},{1,2}]]

func_list_4_box_sp = [coeff_func_for_part(4,this_nc_part) for this_nc_part in nc_partitions_4]

elements with one internal edge

In [16]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,2)]),coeff_func_for_part(4,[],[(1,3)])]

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(2,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{1,3}],[(0,1)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{1,2,3}],[(0,1)])

func_list_4_box_sp += [coeff_func_for_part(4,[{0,1},{2,3}],[(1,2)]),coeff_func_for_part(4,[{0,3},{1,2}],[(0,1)])]

elements with two internal edges

In [17]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(1,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(0,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(1,3)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(2,3)]),coeff_func_for_part(4,[],[(0,3),(1,2)])]

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3),(0,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3),(2,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,2),(2,3)])

func_list_4_box_sp += [coeff_func_for_part(4,[{0,2}],[(1,2),(2,3)]),coeff_func_for_part(4,[{1,3}],[(0,1),(1,2)])]

elements with three internal edges

In [18]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(0,3),(1,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,3),(2,3),(1,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(0,2),(0,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3),(0,2),(2,3)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,2),(0,3),(1,2)]),coeff_func_for_part(4,[],[(0,1),(1,3),(2,3)])]

func_list_4_box_sp += [coeff_func_for_part(4,[],[(1,3),(0,3),(1,2)]),coeff_func_for_part(4,[],[(0,1),(0,2),(2,3)])]

elements with four internal edges

In [19]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,3),(2,3),(1,2),(1,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,3),(2,3),(1,2),(0,2)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(0,3)]),coeff_func_for_part(4,[],[],[(0,0),(1,0),(2,0),(3,0)],[],1)]

elements with five internal edges

In [20]:
func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(0,3),(1,3)]),coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(0,3),(0,2)])]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_rotated_4([],[(0,1)],[(0,0),(1,0),(2,0),(3,0)],1)

elements with six internal edges

In [21]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_rotated_4([],[(0,1),(1,2)],[(0,0),(1,0),(2,0),(3,0)],1)

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(2,3)],[(0,0),(1,0),(2,0),(3,0)],[],1),coeff_func_for_part(4,[],[(0,3),(1,2)],[(0,0),(1,0),(2,0),(3,0)],[],1)]

elements with seven internal edges

In [22]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_rotated_4([],[(0,1),(1,2),(2,3)],[(0,0),(1,0),(2,0),(3,0)],1)

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

elements with eight internal edges

In [23]:
func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(3,0)],[(0,0),(1,0),(2,0),(3,0)],[],1)]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)

elements with nine internal edges

In [24]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(1,1),(2,1),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(2,3)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[(0,1),(2,3)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

func_list_4_box_sp += [coeff_func_for_part(4,[],[(1,2),(0,3)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[(1,2),(0,3)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

elements with ten internal edges

In [25]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2),(2,3)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2),(2,3)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,1),(0,2),(1,0),(2,0),(2,1),(2,2),(3,2)],[(0,1),(1,2)],3),coeff_func_for_part(4,[],[],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,1),(3,2)],[(0,1),(1,2)],3)]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(1,0),(1,1),(2,1),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(0,1),(1,1),(2,1),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

elements with 11 internal edges

In [26]:
func_list_4_box_sp  += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(3,0)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(3,0)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(0,1),(0,2),(1,0),(2,0),(2,1),(2,2),(3,2)],[(0,1),(1,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,1),(3,2)],[(0,1),(1,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,3)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(1,2)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(2,3)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

elements with 12 internal edges

In [27]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,3),(1,0),(1,1),(2,1),(2,2),(3,2),(3,3)],[(0,1),(1,2),(2,3),(3,0)],4)]

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,1),(1,0),(1,2),(2,2),(2,3),(3,1),(3,3)],[(0,1),(0,2),(1,3),(2,3)],4)]

elements with 13 internal edges

In [28]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(0,3),(1,0),(1,1),(2,1),(2,2),(3,2),(3,3)],[(0,1),(1,2),(2,3),(3,0)],4)

a few more odd ones with 4 internal vertices

In [29]:
func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,1),(1,0),(1,2),(2,2),(2,3),(3,1),(3,3)],[(0,1),(0,2),(1,2),(1,3)],4)]

func_list_4_box_sp += [coeff_func_for_part(4, [], [], [(0, 0), (0, 1), (1, 0), (1, 2), (2, 2), (2, 3), (3, 1), (3, 3)], [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)], 4)]

func_list_4_box_sp += [coeff_func_for_part(4, [], [(0, 1)], [(0, 0), (0, 1), (1, 0), (1, 2), (2, 2), (2, 3), (3, 1), (3, 3)], [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)], 4)]

total number of elements

In [30]:
print(f"We have {len(func_list_4_box_sp)} diagram elements")

We have 204 diagram elements


Get the dimension of 4-box space spanned by the diagram elements

In [31]:
def myfunc3(x):
    return [func(x[0],x[1],x[2],x[3]) for func in func_list_4_box_sp]

if __name__ == '__main__':
    with mp.Pool() as p:
        Ap_array = p.map(myfunc3, quad_vert_orb_rep)

In [32]:
# want 134 for O^-(6,3)
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 116, spanned by 204 elements


In [33]:
raise Exception("Stop here")

Exception: Stop here

## compute product of diagram elements

Get the list of generator indexes for an automorphism mapping one vertex to another

In [34]:
def try_next_orb(j, i_list, S, G_gens):
    for next_i_cand in range(0, len(G_gens)):
        j_cand = G_gens[next_i_cand](j)
        if j_cand not in S:
            S_prime = S
            S_prime.add(j_cand)
            i_prime_list = i_list + [next_i_cand]
            return (True, j_cand, i_prime_list, S_prime)
    return (False, j, i_list, S)

def try_replace_leaf(i_list, S, orbit_pres, G_gens):
    if len(i_list) == 0:
        return (False, 0, i_list, S)
    i_branch = i_list[0:-1]
    i_last_term = i_list[-1]
    if i_last_term == len(G_gens)-1:
        return (False, 0, i_list, S)
    j_at_branch = [it[0] for it in orbit_pres if it[1] == i_branch][0]
    for i_last_cand in range(i_last_term + 1, len(G_gens)):
        j_cand = G_gens[i_last_cand](j_at_branch)
        if j_cand not in S:
            S_prime = S
            S_prime.add(j_cand)
            i_prime_list = i_branch + [i_last_cand]
            return (True, j_cand, i_prime_list, S_prime)
    return (False, 0, i_list, S)

def orbit(G_gens, init_pt = 0):
    orbit_pres = [(init_pt,[])]
    orbit_set = {init_pt}
    cont_flag = True
    while cont_flag:
        last_j = orbit_pres[-1][0]
        last_i_list = orbit_pres[-1][1]
        tno_suc, next_j, next_i_list, orbit_set = try_next_orb(last_j, last_i_list, orbit_set, G_gens)
        if tno_suc == True:
            orbit_pres.append((next_j, next_i_list))
        else:
            cont_flag_2 = True
            while cont_flag_2:
                trl_suc, next_j, next_i_list, orbit_set = try_replace_leaf(last_i_list, orbit_set, orbit_pres, G_gens)
                if trl_suc == True:
                    orbit_pres.append((next_j, next_i_list))
                    cont_flag_2 = False
                else:
                    if len(last_i_list) <= 1:
                        cont_flag = False
                        break
                    else:
                        last_i_list = last_i_list[0:-1]
    return (orbit_pres, orbit_set)

In [35]:
G = the_graph.automorphism_group()
orbit_pres, orbit_set = orbit(G.gens())
print(f"We have an orbit of length {len(orbit_set)}, while we have {num_vert} vertices")

We have an orbit of length 112, while we have 112 vertices


In [36]:
from functools import reduce

In [37]:
def aut_elem_from(G_gens, a, b):
    orbit_pres, orbit_set = orbit(G_gens, init_pt = a)
    ind_list_for_b = [it[1] for it in orbit_pres if it[0] == b][0]
    aut_from_a_to_b = reduce(lambda x, y: x * G_gens[y], ind_list_for_b, G.identity())
    return aut_from_a_to_b

In [38]:
def d_in_graph(x, y):
    if x == y:
        return 0
    if y in the_graph.neighbors(x):
        return 1
    return 2

def trip_orb_rep_for(a, b, c):
    for tvor in trip_vert_orb_rep:
        a_p, b_p, c_p = tvor
        if d_in_graph(a, b) == d_in_graph(a_p, b_p) and \
        d_in_graph(a, c) == d_in_graph(a_p, c_p) and \
        d_in_graph(b, c) == d_in_graph(b_p, c_p):
            return tvor
    return False

Find orbit representative for a given quadruple of vertexes

In [39]:
def quad_orb_rep_for(a, b, c, d):
    trip_orb_rep = trip_orb_rep_for(a, b, c)
    a_p, b_p, c_p = trip_orb_rep
    G1 = the_graph.automorphism_group()
    G1_gens = G1.gens()
    g1 = aut_elem_from(G1_gens, a, a_p)
    G2 = the_graph.automorphism_group(partition=[[a_p], list(range(0,a_p)) + list(range(a_p+1, num_vert))])
    G2_gens = G2.gens()
    g2 = aut_elem_from(G2_gens, g1(b), b_p)
    compl_set = set(range(0, num_vert))
    compl_set.remove(a_p)
    compl_set.discard(b_p)
    my_part = [[a_p], [b_p], list(compl_set)]
    if a_p == b_p:
        my_part = [[a_p], list(compl_set)]
    G3 = the_graph.automorphism_group(partition=my_part)
    G3_gens = G3.gens()
    g3 = aut_elem_from(G3_gens, (g1 * g2)(c), c_p)
    d_p = (g1 * g2 * g3)(d)
    for quad_orb in quad_vert_orb_rep:
        if trip_orb_rep == quad_orb[0:3] and d_p in quad_vert_orb_fourth_cand[quad_orb]:
            return quad_orb
    return False

## compute multiplicity for product

We want to compute $(D_1 * D_2)(a, b, c, d) = \sum_{e, f} D_1(a, b, e, f) D_2(f, e, c, d)$ by understanding the multiplicity of orbit types for $(a, b, e, f)$ and $(f, e, c, d)$

In [40]:
def prod_mult(a, b, c, d):
    mult_dict = {}
    for e, f in it.product(range(0, num_vert), range(0, num_vert)):
        orb_rep_1 = quad_orb_rep_for(a, b, e, f)
        orb_rep_2 = quad_orb_rep_for(f, e, c, d)
        orb_pair = (orb_rep_1, orb_rep_2)
        if orb_pair in mult_dict.keys():
            old_cnt = mult_dict[orb_pair]
            mult_dict[orb_pair] = old_cnt + 1
        else:
            mult_dict[orb_pair] = 1
    return mult_dict

In [41]:
raise Exception("Stop here")

Exception: Stop here

heavy computation to get multiplication table, parallelized

In [ ]:
# parallel computation, takes long
def myfunc1(orb_rep):
    return (orb_rep, prod_mult(orb_rep[0], orb_rep[1], orb_rep[2], orb_rep[3]))

if __name__ == '__main__':
    with mp.Pool() as p:
        tup_array = p.map(myfunc1, quad_vert_orb_rep)

In [ ]:
raise Exception("Stop here")

In [ ]:
# save the computed multiplication table
with open("tup.pickle", "wb") as fp:
    pickle.dump(tup_array, fp)

use the multiplication table to compute product of diagram elements

In [42]:
# import saved multiplication table
with open("tup.pickle", "rb") as fp:
    tup_array = pickle.load(fp)

In [43]:
# product of list-represented elements
def prod_at(j, k, i):
    mult_dict = tup_array[i][1]
    res = 0
    j_array = [row[j] for row in Ap_array]
    k_array = [row[k] for row in Ap_array]
    for key, value in mult_dict.items():
        a_ind = quad_vert_orb_rep.index(key[0])
        b_ind = quad_vert_orb_rep.index(key[1])
        res += j_array[a_ind] * k_array[b_ind] * value
    return res

In [44]:
# rotation of list-represented elements

def rotated_representative(orb):
    rot_orb = quad_orb_rep_for(orb[3], orb[0], orb[1], orb[2])
    return rot_orb

def rot_rep_ind(orb):
    return quad_vert_orb_rep.index(rotated_representative(orb))

rot_ind_list = []
if __name__ == '__main__':
    with mp.Pool() as p:
        rot_ind_list = p.map(rot_rep_ind, quad_vert_orb_rep)

def rotated_func(fun_list):
    return list(map(lambda x: fun_list[x], rot_ind_list))

In [45]:
def inv_rotated_representative(orb):
    inv_rot_orb = quad_orb_rep_for(orb[1], orb[2], orb[3], orb[0])
    return inv_rot_orb

def inv_rot_rep_ind(orb):
    return quad_vert_orb_rep.index(inv_rotated_representative(orb))

inv_rot_ind_list = []
if __name__ == '__main__':
    with mp.Pool() as p:
        inv_rot_ind_list = p.map(inv_rot_rep_ind, quad_vert_orb_rep)

def inv_rotated_func(fun_list):
    return list(map(lambda x: fun_list[x], inv_rot_ind_list))

In [46]:
def twice_rotated_representative(orb):
    twice_rot_orb = quad_orb_rep_for(orb[2], orb[3], orb[0], orb[1])
    return twice_rot_orb

def twice_rot_rep_ind(orb):
    return quad_vert_orb_rep.index(twice_rotated_representative(orb))

twice_rot_ind_list = []
if __name__ == '__main__':
    with mp.Pool() as p:
        twice_rot_ind_list = p.map(twice_rot_rep_ind, quad_vert_orb_rep)

def twice_rotated_func(fun_list):
    return list(map(lambda x: fun_list[x], twice_rot_ind_list))

In [47]:
# start with product of last 10 elements with themselves

old_rank = np.linalg.matrix_rank(np.array(Ap_array))
for j in range(195,199):
    for k in range(195,199):
        bpr_array = [prod_at(j,k,i) for i in range(len(quad_vert_orb_rep))]
        newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
        new_rank = np.linalg.matrix_rank(np.array(newAp_array))
        if old_rank < new_rank:
            print(f"Found a new element as the product of {j}-th and {k}-th element.")
            Ap_array = newAp_array
            old_rank = new_rank

Found a new element as the product of 195-th and 195-th element.
Found a new element as the product of 195-th and 198-th element.


In [48]:
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 118, spanned by 206 elements


check rotated elements

In [49]:
old_rank = np.linalg.matrix_rank(np.array(Ap_array))
for j in range(204,207):
    j_array = [Ap_array[i][j] for i in range(len(quad_vert_orb_rep))]
    bpr_array = rotated_func(j_array)
    newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
    new_rank = np.linalg.matrix_rank(np.array(newAp_array))
    if old_rank < new_rank:
        print(f"Found a new element as the rotation of {j}-th element.")
        Ap_array = newAp_array
        old_rank = new_rank
    bpr_array = inv_rotated_func(j_array)
    newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
    new_rank = np.linalg.matrix_rank(np.array(newAp_array))
    if old_rank < new_rank:
        print(f"Found a new element as the inverse rotation of {j}-th element.")
        Ap_array = newAp_array
        old_rank = new_rank
    bpr_array = twice_rotated_func(j_array)
    newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
    new_rank = np.linalg.matrix_rank(np.array(newAp_array))
    if old_rank < new_rank:
        print(f"Found a new element as 180-deg rotation of {j}-th element.")
        Ap_array = newAp_array
        old_rank = new_rank

Found a new element as the rotation of 204-th element.
Found a new element as the rotation of 205-th element.


In [50]:
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 120, spanned by 208 elements


In [51]:
# check product again
old_rank = np.linalg.matrix_rank(np.array(Ap_array))
for j in range(195,208):
    for k in range(204,208):
        bpr_array = [prod_at(j,k,i) for i in range(len(quad_vert_orb_rep))]
        newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
        new_rank = np.linalg.matrix_rank(np.array(newAp_array))
        if old_rank < new_rank:
            print(f"Found a new element as the product of {j}-th and {k}-th element.")
            Ap_array = newAp_array
            old_rank = new_rank

Found a new element as the product of 195-th and 206-th element.
Found a new element as the product of 195-th and 207-th element.
Found a new element as the product of 198-th and 207-th element.
Found a new element as the product of 200-th and 207-th element.
Found a new element as the product of 201-th and 207-th element.
Found a new element as the product of 207-th and 205-th element.
Found a new element as the product of 207-th and 207-th element.


In [52]:
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 127, spanned by 215 elements


In [53]:
# check rotation again
old_rank = np.linalg.matrix_rank(np.array(Ap_array))
for j in range(207,215):
    j_array = [Ap_array[i][j] for i in range(len(quad_vert_orb_rep))]
    bpr_array = rotated_func(j_array)
    newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
    new_rank = np.linalg.matrix_rank(np.array(newAp_array))
    if old_rank < new_rank:
        print(f"Found a new element as the rotation of {j}-th element.")
        Ap_array = newAp_array
        old_rank = new_rank
    bpr_array = inv_rotated_func(j_array)
    newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
    new_rank = np.linalg.matrix_rank(np.array(newAp_array))
    if old_rank < new_rank:
        print(f"Found a new element as the inverse rotation of {j}-th element.")
        Ap_array = newAp_array
        old_rank = new_rank
    bpr_array = twice_rotated_func(j_array)
    newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
    new_rank = np.linalg.matrix_rank(np.array(newAp_array))
    if old_rank < new_rank:
        print(f"Found a new element as 180-deg rotation of {j}-th element.")
        Ap_array = newAp_array
        old_rank = new_rank

Found a new element as the rotation of 209-th element.
Found a new element as the rotation of 210-th element.
Found a new element as the inverse rotation of 212-th element.


In [54]:
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 130, spanned by 218 elements


In [55]:
# check product yet again
old_rank = np.linalg.matrix_rank(np.array(Ap_array))
for j in range(65, 68):
    for k in range(209,218):
        bpr_array = [prod_at(j,k,i) for i in range(len(quad_vert_orb_rep))]
        newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]
        new_rank = np.linalg.matrix_rank(np.array(newAp_array))
        if old_rank < new_rank:
            print(f"Found a new element as the product of {j}-th and {k}-th element.")
            Ap_array = newAp_array
            old_rank = new_rank

Found a new element as the product of 65-th and 217-th element.
Found a new element as the product of 67-th and 216-th element.


In [56]:
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 132, spanned by 220 elements


In [ ]:
raise Exception("Stop here")

In [57]:
# the square of 213-th element has components too big,
# which gives a type error in numpy that relies on 64-bit integers.
# check it using sympy instead
bpr_array = [prod_at(213,213,i) for i in range(len(quad_vert_orb_rep))]
newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]

In [58]:
sp.Matrix(newAp_array).rank()

133

In [59]:
Ap_array = newAp_array

In [60]:
# same with 213-th times 214-th
bpr_array = [prod_at(213,214,i) for i in range(len(quad_vert_orb_rep))]
newAp_array = [Ap_array[i] + [bpr_array[i]] for i in range(0,len(Ap_array))]

In [61]:
# now we expect rank 134
sp.Matrix(newAp_array).rank()

134

In [62]:
Ap_array = newAp_array

## Finding flip

check if the flip permutation is in the linear span of diagram elements

In [63]:
def func_4_perm(x,y,z,w):
    if x == z and y == w:
        return 1
    return 0

def myfunc4(x):
    return func_4_perm(x[0],x[1],x[2],x[3])

In [64]:
bp_array = [myfunc4(x) for x in quad_vert_orb_rep]

In [65]:
myvars = [sp.Symbol("x%d" % i) for i in range(len(Ap_array[0]))]

In [66]:
sols = sp.linsolve((sp.Matrix(Ap_array),sp.Matrix(bp_array)),myvars); sols

{(-4*x201 - 21025310690896414103783342291/5264138895030288000000, -6*x129 - 6*x201 - 775159348760990269703359247/87735648250504800000, 3783980408877756001776041/259957476297792000000 - 6*x130, -6*x127 - 124822837096765253849051885573/438678241252524000000, -6*x128 - 6*x201 + 6318600791316477269346343/179970560513856000000, -20*x152 - 6*x201 + 5677343781809041942707629/233961728668012800000, 5603735832572982531374861/389936214446688000000 - 20*x151, -14*x129 - 14*x130 - 40*x151 - 20*x160 - 20*x163 + 608087230517011462976629079/21056555580121152000000, -14*x127 - 14*x130 - 40*x152 - 20*x157 - 20*x164 - 40*x201 - 1690603723396487842966226953/21056555580121152000000, -14*x127 - 14*x128 - 40*x151 - 20*x158 - 20*x161 - 2453752911417442177423289467/21056555580121152000000, -14*x128 - 14*x129 - 40*x152 - 20*x159 - 20*x162 - 8*x201 + 396860824441889550160649099/3008079368588736000000, -32*x127 - 32*x128 - 32*x129 - 32*x130 - 80*x151 - 80*x152 - 40*x157 - 40*x158 - 40*x159 - 40*x160 - 40*x161 - 

In [67]:
# substitute free variable by 0
sol_cand = list(sols)[0].subs([(thevar, 0) for thevar in myvars]); sol_cand

(-21025310690896414103783342291/5264138895030288000000, -775159348760990269703359247/87735648250504800000, 3783980408877756001776041/259957476297792000000, -124822837096765253849051885573/438678241252524000000, 6318600791316477269346343/179970560513856000000, 5677343781809041942707629/233961728668012800000, 5603735832572982531374861/389936214446688000000, 608087230517011462976629079/21056555580121152000000, -1690603723396487842966226953/21056555580121152000000, -2453752911417442177423289467/21056555580121152000000, 396860824441889550160649099/3008079368588736000000, -28951758557641654774576699/8225217023484825000, -1208618036590724771727470301019/752019842147184000000, -2050005815209742492885835337/5264138895030288000000, -6074004870753698602679941397/1316034723757572000000, -17293567340745128736343483/2339617286680128000000, -9967921877334091734731059/802154498290329600, -22381250680490198221750763/2339617286680128000000, 37660957035372776051657/51420160146816000000, 14020980051657388

In [ ]:
raise Exception("stop here.")